# PySpark part 7

## MLlib

Apache Spark offers a Machine Learning API called MLlib. PySpark has this machine learning API in Python as well. It supports different kind of algorithms, which are mentioned below −

* `mllib.classification`: Library for classification algorithms, such as random forest, naive bayes, decision tree.
* `millib.clustering`: self explanatory
* `mllib.fpm`: **Frequent Pattern Matching** mines for frequent items, itemsets, subsequences or other substructures
* `millib.linalg`: linear algebra tools
* `millib.recommendation`:collaborative filtering for recommendation systems.
* `spark.millib`: It ¬currently supports model-based collaborative filtering, in which users and products are described by a small set of latent factors that can be used to predict missing entries. spark.mllib uses the Alternating Least Squares (ALS) algorithm to learn these latent factors. ..

**Wtf does this mean??** -> [How do you build a “People who bought this also bought that”-style recommendation engine](https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/)
* `millib.regression`: family of regression algorithms, including linear.

### Example

The following example is of collaborative filtering using ALS algorithm to build the recommendation model and evaluate it on training data.


In [1]:
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating


sc = SparkContext('local', 'mllib app') # initialize the app
data = sc.textFile('test.csv') # reading in data
# then split into columns / matrix, let's see the outcome using .collect()
data.map(lambda l: l.split(',')).collect()

[['1', '1', '5.0'],
 ['1', '2', '1.0'],
 ['1', '3', '5.0'],
 ['1', '4', '1.0'],
 ['2', '1', '5.0'],
 ['2', '2', '1.0'],
 ['2', '3', '5.0'],
 ['2', '4', '1.0'],
 ['3', '1', '1.0'],
 ['3', '2', '5.0'],
 ['3', '3', '1.0'],
 ['3', '4', '5.0'],
 ['4', '1', '1.0'],
 ['4', '2', '5.0'],
 ['4', '3', '1.0'],
 ['4', '4', '5.0']]

**That is the first 3 columns of the data**

* 1 column represents: user id
* 2 column represents: product id
* 3 column represents: rating

Now Let's apply map function again, to wrap each data point(row) into MLlib Rating class. That looks like...

In [2]:
ratings = data.map(lambda l: l.split(','))\
      .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

# let's take a look at the data
ratings.collect()


[Rating(user=1, product=1, rating=5.0),
 Rating(user=1, product=2, rating=1.0),
 Rating(user=1, product=3, rating=5.0),
 Rating(user=1, product=4, rating=1.0),
 Rating(user=2, product=1, rating=5.0),
 Rating(user=2, product=2, rating=1.0),
 Rating(user=2, product=3, rating=5.0),
 Rating(user=2, product=4, rating=1.0),
 Rating(user=3, product=1, rating=1.0),
 Rating(user=3, product=2, rating=5.0),
 Rating(user=3, product=3, rating=1.0),
 Rating(user=3, product=4, rating=5.0),
 Rating(user=4, product=1, rating=1.0),
 Rating(user=4, product=2, rating=5.0),
 Rating(user=4, product=3, rating=1.0),
 Rating(user=4, product=4, rating=5.0)]

In [3]:
## Looks good!! now we move on to build a model using 
## Alternating Least Squares (ALS)
rank = 10
numIteration = 10
model = ALS.train(ratings, rank, numIteration)
# ratings: RDD of Rating class tuple, rank: number of features to use

# Evaluate the model on training data
## first extract user id and product id only.
testdata = ratings.map(lambda p: (p[0], p[1]))
prediction = model.predictAll(testdata)
# let's see the output of prediction
prediction.collect()

[Rating(user=4, product=4, rating=4.996066843190942),
 Rating(user=4, product=1, rating=1.00118534493132),
 Rating(user=4, product=3, rating=1.00118534493132),
 Rating(user=4, product=2, rating=4.996066843190942),
 Rating(user=1, product=4, rating=1.0010165607980466),
 Rating(user=1, product=1, rating=4.996910639807666),
 Rating(user=1, product=3, rating=4.996910639807666),
 Rating(user=1, product=2, rating=1.0010165607980466),
 Rating(user=3, product=4, rating=4.996066843190942),
 Rating(user=3, product=1, rating=1.00118534493132),
 Rating(user=3, product=3, rating=1.00118534493132),
 Rating(user=3, product=2, rating=4.996066843190942),
 Rating(user=2, product=4, rating=1.0010165607980466),
 Rating(user=2, product=1, rating=4.996910639807666),
 Rating(user=2, product=3, rating=4.996910639807666),
 Rating(user=2, product=2, rating=1.0010165607980466)]

In [4]:
## Now that is a list of rating class with predictions. 
# We will map this into list of nested tuples.
prediction = prediction.map(lambda r: ((r[0], r[1]), r[2]))

#which outputs..
prediction.collect()

[((4, 4), 4.996066843190942),
 ((4, 1), 1.00118534493132),
 ((4, 3), 1.00118534493132),
 ((4, 2), 4.996066843190942),
 ((1, 4), 1.0010165607980466),
 ((1, 1), 4.996910639807666),
 ((1, 3), 4.996910639807666),
 ((1, 2), 1.0010165607980466),
 ((3, 4), 4.996066843190942),
 ((3, 1), 1.00118534493132),
 ((3, 3), 1.00118534493132),
 ((3, 2), 4.996066843190942),
 ((2, 4), 1.0010165607980466),
 ((2, 1), 4.996910639807666),
 ((2, 3), 4.996910639807666),
 ((2, 2), 1.0010165607980466)]

In [6]:
## what is this??
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(prediction)

# calculate MSE
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 6.8630768362371275e-06


In [ ]:
## Now you can save and load your model like...

model.save(sc, "path/to/your/model/file")
   sameModel = MatrixFactorizationModel.load(sc, "where/your/model/is")